# Parsa KafshduziBukani
This project develops solvers for a Sokoban-like game with portals. The goal is to move boxes to their specific goals using a player, dealing with walls and up to two portal pairs.

We implemented four search algorithms: BFS, DFS, IDS, and A*. These find paths to solve the game efficiently.

In [1]:
from game import Game
import time
import os
import re

## Load maps

In [51]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Solvers

### Game State and Solver Classes

 State Class:

The `State` class represents the current state of the game. It stores the positions of the player and boxes, along with the moves made so far. Each state has three instance variables:

- **`player_pos`**: A tuple of two integers representing the player’s row and column position.  
- **`boxes_pos`**: A tuple of tuples, where each inner tuple contains two integers for a box’s row and column position.  
- **`solution_path`**: A string of move directions, initially empty, tracking the path taken.  

  
       
 GameSolver Class:

The `GameSolver` class manages the game and generates possible moves. It simplifies how solvers interact with the game map. It has three main instance variables:

- **`game`**: An instance of the `Game` class, holding the map and current game details.  
- **`initial_state`**: A `State` object set at the start, storing the initial player and box positions.  
- **`goal_positions`**: A tuple of tuples, where each inner tuple contains two integers for a goal’s row and column position.  


In [52]:
from enum import Enum
from dataclasses import dataclass
from typing import List, Tuple, Set, Dict

class Direction(Enum):
    UP = 'U'
    DOWN = 'D'
    LEFT = 'L'
    RIGHT = 'R'

@dataclass(frozen=True)
class State:
    player_pos: Tuple[int, int]
    boxes_pos: Tuple[Tuple[int, int], ...]
    solution_path: str = ""
    
    def get_player_pos(self) -> Tuple[int, int]:
        return self.player_pos
    
    def get_boxes_pos(self) -> Tuple[Tuple[int, int], ...]:
        return self.boxes_pos
    
    def get_solution_path(self) -> str:
        return self.solution_path
    
    def __eq__(self, other: object) -> bool:
        if not isinstance(other, State):
            return False
        return self.player_pos == other.player_pos and self.boxes_pos == other.boxes_pos
    
    def __hash__(self) -> int:
        return hash((self.player_pos, self.boxes_pos))
    
    def __lt__(self, other: "State") -> bool:
        return True 

class GameSolver:
    def __init__(self, game: Game):
        self.game = game
        self.initial_state = self._build_initial_state()
        self.goal_positions = tuple(tuple(goal) for goal in game.get_goal_locations())

    def _build_initial_state(self) -> State:
        return State(
            player_pos=self.game.get_player_position(),
            boxes_pos=tuple(tuple(box) for box in self.game.get_box_locations()),
            solution_path=""
        )

    def _apply_move_and_get_state(self, state: State, direction: str) -> State | None:
        self.game.set_player_position(state.get_player_pos())
        self.game.set_box_positions(list(state.get_boxes_pos()))
        
        if self.game.apply_move(direction):
            return State(
                player_pos=self.game.get_player_position(),
                boxes_pos=tuple(tuple(box) for box in self.game.get_box_locations()),
                solution_path=state.get_solution_path() + direction
            )
        return None

    def get_next_states(self, state: State, visited: Set[State]) -> List[State]:
        next_states = []
        for direction in Direction:
            new_state = self._apply_move_and_get_state(state, direction.value)
            if new_state and new_state not in visited:
                next_states.append(new_state)
        return next_states

    def get_next_dls_astar_states(self, state: State, depth: int, visited: Dict[State, int]) -> List[State]:
        next_states = []
        for direction in Direction:
            new_state = self._apply_move_and_get_state(state, direction.value)
            if new_state and (new_state not in visited or visited[new_state] > depth + 1):
                next_states.append(new_state)
        return next_states
  
    def is_goal_state(self, state: State) -> bool:
        boxes = state.get_boxes_pos()
        for i in range(len(self.goal_positions)):
            if boxes[i] != self.goal_positions[i]:
                return False
        return True

    def get_initial_state(self) -> State:
        return self.initial_state

### BFS Solver

The `solver_bfs` function uses **Breadth-First Search (BFS)** to find a solution for the game. It explores all possible moves level by level to ensure the shortest path.

Key parts:

- **Queue**: Uses a `deque` to store states, adding new ones to the end and taking from the front.  
- **Visited States**: A `set` tracks seen states to avoid revisiting them.  
- **Next States**: Calls `GameSolver.get_next_states` to generate moves from the current state.  
- **Goal Check**: Uses `GameSolver.is_goal_state` to stop when boxes reach their goal positions.  
- **Return**: Outputs the solution path and the number of visited states.  

In [53]:
from collections import deque

def solver_bfs(game_map):
    solver = GameSolver(Game(game_map))
    if solver.is_goal_state(solver.get_initial_state()):
        return "", 0
    
    queue = deque([solver.get_initial_state()])
    visited_states = {solver.get_initial_state()}

    while queue:
        cur_state = queue.popleft()
        for next_state in solver.get_next_states(cur_state, visited_states):
            if solver.is_goal_state(next_state):
                return next_state.get_solution_path(), len(visited_states)
            queue.append(next_state)
            visited_states.add(next_state)

    return None, len(visited_states)

### DFS Solver

The `solver_dfs` function uses **Depth-First Search (DFS)** to find a solution for the game. It explores deep into one path before backtracking.

Key parts:

- **Stack**: Uses a `deque` as a stack, adding and removing states from the end.  
- **Visited States**: A `set` tracks seen states to avoid revisiting them.  
- **Next States**: Calls `GameSolver.get_next_states` to generate moves from the current state.  
- **Goal Check**: Uses `GameSolver.is_goal_state` to stop when boxes reach their goal positions.  
- **Return**: Outputs the solution path and the stack length (states in the current path).  

In [54]:
from collections import deque

def solver_dfs(game_map):
    solver = GameSolver(Game(game_map))
    goal_positions = solver.goal_positions 
    stack = deque()
    visited_states = set()

    if solver.is_goal_state(solver.get_initial_state()):
        return "", 0
    initial_state = solver.get_initial_state()
    stack.append(initial_state)
    visited_states.add(initial_state)

    while stack:
        cur_state = stack[-1]
        next_States = solver.get_next_states(cur_state, visited_states)

        if len(next_States) == 0:
            stack.pop()

        for next_state in next_States:
            if solver.is_goal_state(next_state):
                return next_state.get_solution_path(), len(stack)
            
            stack.append(next_state)
            visited_states.add(next_state)

    return None, len(visited_states)

### IDS Solver

The `solver_ids` function uses **Iterative Deepening Search (IDS)** to find a solution. It repeatedly runs **Depth-Limited Search (DLS)** with increasing depth limits.

Key parts:

##### **Depth-Limited Search (DLS)**
- **Stack**: Uses a `deque` to store states, popping from the end.  
- **Visited States**: A `dict` tracks states and their depths to enforce the limit.  
- **Next States**: Calls `GameSolver.get_next_dls_astar_states` to generate moves.  
- **Return**: Outputs the solution path, `len(stack)`, and a status:  
  - `1` → Solution found  
  - `2` → No solution exists  
  - `0` → Continue searching  

##### **Iterative Deepening Loop**
- **Loop**: Increases depth and calls DLS until a solution is found or no states remain.  
- **Seen States**: Sums up `visited states` from each DLS run to track total states explored in the path to reaching the goal.  


In [55]:
def DLS(solver: GameSolver, limit, goal_positions, initial_state): 
    stack = deque()
    visited_states = dict()

    stack.append(initial_state)
    visited_states[initial_state] = 0
    is_without_solution = True

    while stack:
        cur_state = stack[-1]
        depth = visited_states[cur_state]
        next_States = solver.get_next_dls_astar_states(cur_state, depth, visited_states)

        if len(next_States) == 0 or (depth >= limit):
            if len(next_States) != 0:
                is_without_solution = False
            
            stack.pop()
            continue

        for next_state in next_States:
            if solver.is_goal_state(next_state):
                return next_state.get_solution_path(), len(stack), 1

            stack.append(next_state)
            visited_states[next_state] = depth + 1

    return None, len(visited_states), (2 if is_without_solution else 0)


def solver_ids(game_map):
    HAS_SOLUTION = 1
    NO_SOLUTION = 2

    solver = GameSolver(Game(game_map))
    goal_positions = solver.goal_positions

    if solver.is_goal_state(solver.get_initial_state()):
        return "", 0
    initial_state = solver.get_initial_state()
    
    seen_states_sum = 0
    depth = 0
    while True:
        solution_path, num_of_states, answer = DLS(solver, depth, goal_positions, initial_state)
        seen_states_sum += num_of_states
        if answer == HAS_SOLUTION:
            return solution_path, seen_states_sum
        elif answer == NO_SOLUTION:
            break
         
        depth += 1
    
    return None, 0

### Heuristic Functions

Two heuristic functions for **A\*** have been implemented to estimate the cost of solving the game and guide the search.

#### **1. Manhattan Heuristic**

This function calculates a basic cost using **Manhattan distances**:       
        
- **Box-to-Goal**: Sums the Manhattan distance (x and y differences) from each box to its goal.  
- **Player-to-Box**: Finds the smallest Manhattan distance from the player to any box.  
- **Total Cost**: Adds the box-to-goal sum and the minimum player-to-box distance.  

*Notes*:  
  - This heuristic **ignores portals and walls**, making it simple but less accurate.  
  - Player's distance to all boxes does not matter, but the nearest box to it is important.  

#### **2. Portal-Deadlock-Goal Heuristic**

This function improves accuracy by considering **portals, walls, and goals**:

- **Box-to-Goal**: Uses `get_minimum_distance_with_portals` to compute the minimum distance from each box to its goal.  
- **Player-to-Box**: Finds the smallest distance from the player to any unsolved box.  
- **Wall Penalty**: Adds `+2` to the cost for each box near a wall, avoiding deadlocks.  
- **Goal Bonus**: Subtracts `-2` for each box already on its goal, rewarding progress.  
- **Total Cost**: Combines box-to-goal cost, wall penalties, goal bonuses, and minimum player-to-box distance.  


In [56]:
def manhattan_heuristic(state: State, goal_positions: Tuple[Tuple[int, int], ...], portals, game: Game) -> int:
    total_distance = 0
    boxes = state.get_boxes_pos()
    player_x, player_y = state.get_player_pos()

    min_player_to_box = float('inf')

    for i in range(len(boxes)):
        box_x, box_y = boxes[i]
        goal_x, goal_y = goal_positions[i]

        box_to_goal_dist = abs(box_x - goal_x) + abs(box_y - goal_y)
        total_distance += box_to_goal_dist

        player_to_box_dist = abs(player_x - box_x) + abs(player_y - box_y)
        min_player_to_box = min(min_player_to_box, player_to_box_dist)

    return total_distance + min_player_to_box


def get_minimum_distance_with_portals(start, end, portals):
    """
    Computes the minimum Manhattan distance from 'start' to 'end',
    considering direct paths and paths using up to two portal pairs.
    """
    direct_distance = abs(start[0] - end[0]) + abs(start[1] - end[1])
    min_distance = direct_distance

    for p1_in, p1_out in portals:
        dist_via_p1 = (abs(start[0] - p1_in[0]) + abs(start[1] - p1_in[1]) + 
                       abs(p1_out[0] - end[0]) + abs(p1_out[1] - end[1]))
        
        min_distance = min(min_distance, dist_via_p1)

    if len(portals) >= 2:
        p1_in, p1_out = portals[0]
        p2_in, p2_out = portals[1]
        
        dist_p1_then_p2 = (abs(start[0] - p1_in[0]) + abs(start[1] - p1_in[1]) + 
                           abs(p1_out[0] - p2_in[0]) + abs(p1_out[1] - p2_in[1]) + 
                           abs(p2_out[0] - end[0]) + abs(p2_out[1] - end[1]))
        
        dist_p2_then_p1 = (abs(start[0] - p2_in[0]) + abs(start[1] - p2_in[1]) + 
                           abs(p2_out[0] - p1_in[0]) + abs(p2_out[1] - p1_in[1]) + 
                           abs(p1_out[0] - end[0]) + abs(p1_out[1] - end[1]))
        
        min_distance = min(min_distance, dist_p1_then_p2, dist_p2_then_p1)

    return min_distance

def portal_deadlock_goals_heuristic(state: State, goal_positions: Tuple[Tuple[int, int], ...], portals, game: Game) -> int:
    boxes = state.get_boxes_pos()
    player_pos = state.get_player_pos()
    total_cost = 0
    min_player_to_box = float('inf')
    
    for i, box in enumerate(boxes):
        if box != goal_positions[i]: 
            player_to_box = get_minimum_distance_with_portals(player_pos, box, portals)
            min_player_to_box = min(player_to_box, min_player_to_box)

            box_to_goal = get_minimum_distance_with_portals(box, goal_positions[i], portals)
            total_cost += box_to_goal

            if any("W" in game.get_elements_in_position((box[0] + dy, box[1] + dx))
                   for dy, dx in [(1, 0), (-1, 0), (0, 1), (0, -1)]):
                total_cost += 2
        else: 
            total_cost -= 2
    
    return total_cost + min_player_to_box

### A* Solver

Two versions of the solver_astar function have been implemented.

#### **solver_astar:**

This is a faster version of A* that prioritizes quick solutions.

Key parts:
- **Heap**: Uses a ```min_heap``` to prioritize states by cost plus weighted heuristic.
- **Visited States**: A dict tracks states and their costs, skipping if a higher cost is revisited.
- **Next States**: Calls ```GameSolver.get_next_dls_astar_states``` to generate moves.
- **Goal Check**: Checks each new state with ```GameSolver.is_goal_state``` and returns immediately if a goal is found, ignoring lower-cost states in the heap.
- **Return**: Gives the solution path and number of visited states.

#### **solver_astar_optimal:**

This version ensures the shortest path _(if the heuristic function is admissible)_ by selecting the best state.

Key parts:
- **Heap**: Uses a ```min_heap``` with the same priority logic.
- **Visited States**: A dict tracks costs, skipping higher-cost revisits.
- **Is Goal State**: A dict tracks which states are goals, precomputed for the initial state.
- **Next States**: Calls ```GameSolver.get_next_dls_astar_states``` to generate moves.
- **Goal Check**: Checks new states with ```GameSolver.is_goal_state```. If a goal is found, returns only if its cost is less than or equal to the heap’s top priority, ensuring the shortest path.
- **Return**: Gives the solution path and number of visited states.

**Differences**:
- ```solver_astar``` is faster because it returns the first goal state it finds, even if a shorter path exists in the heap.
- ```solver_astar_optimal``` guarantees the shortest path (with an admissible heuristic like ```portal_deadlock_goals_heuristic```) by only returning the goal state with the lowest cost, checked against the heap’s top priority.
- The trade-off is that ```solver_astar_optimal``` may take longer due to waiting for the optimal state, while ```solver_astar``` sacrifices optimality for speed.  


**Notes:**
  *  In this project the **solver_astar** function is used for answering the questions.
  *  The **solver_astar** function is optimal with ```weight <= 0.5``` for the given maps (Lower weight guarantees more optimality).
  *  The datasets used for **BFS** and **A\*** are different. BFS uses a queue while A* uses a priority queue. In general, queues are much faster than priority queues (eg. Dequeue() is O(1) vs O(log n)). so there might be unexpected timing differnces between the two.
  *  It is also important to note that since the graph in this project is unweighted, **solver_bfs** may well outperform **solver_astar_optimal**.

In [57]:
import heapq

def solver_astar(game_map, heuristic_func=portal_deadlock_goals_heuristic, weight=1):
    solver = GameSolver(Game(game_map))
    if solver.is_goal_state(solver.get_initial_state()):
        return "", 0

    goal_positions = solver.goal_positions
    portals = solver.game.get_portal_locations()
    visited_states = {}
    min_heap = []

    initial_state = solver.get_initial_state()
    initial_cost = 0
    initial_priority = initial_cost + weight * heuristic_func(initial_state, goal_positions, portals, solver.game)
    
    heapq.heappush(min_heap, (initial_priority, initial_cost, initial_state))
    visited_states[initial_state] = initial_cost

    while min_heap:
        _, cur_cost, cur_state = heapq.heappop(min_heap)

        if cur_cost > visited_states.get(cur_state, float('inf')):
            continue

        new_cost = cur_cost + 1
        for next_state in solver.get_next_dls_astar_states(cur_state, cur_cost, visited_states):
            if solver.is_goal_state(next_state):
                return next_state.get_solution_path(), len(visited_states)

            priority = new_cost + weight * heuristic_func(next_state, goal_positions, portals, solver.game)
            heapq.heappush(min_heap, (priority, new_cost, next_state))
            visited_states[next_state] = new_cost

    return None, len(visited_states)


def solver_astar_optimal(game_map, heuristic_func=portal_deadlock_goals_heuristic, weight=1):
    solver = GameSolver(Game(game_map))
    goal_positions = solver.goal_positions
    portals = solver.game.get_portal_locations()
    visited_states = {}
    is_goal_state = {}
    min_heap = []

    initial_state = solver.get_initial_state()
    initial_cost = 0
    initial_priority = initial_cost + weight * heuristic_func(initial_state, goal_positions, portals, solver.game)
    
    heapq.heappush(min_heap, (initial_priority, initial_cost, initial_state))
    visited_states[initial_state] = initial_cost
    is_goal_state[initial_state] = solver.is_goal_state(initial_state)   

    while min_heap:
        _, cur_cost, cur_state = heapq.heappop(min_heap)

        if cur_cost > visited_states.get(cur_state, float('inf')):
            continue

        if is_goal_state[cur_state]:
            return cur_state.get_solution_path(), len(visited_states)

        new_cost = cur_cost + 1
        for next_state in solver.get_next_dls_astar_states(cur_state, cur_cost, visited_states):
            priority = new_cost + weight * heuristic_func(next_state, goal_positions, portals, solver.game)
            
            heapq.heappush(min_heap, (priority, new_cost, next_state))

            if solver.is_goal_state(next_state):
                if priority <= min_heap[0][0]:
                    return next_state.get_solution_path(), len(visited_states)
                is_goal_state[next_state] = True
            else:
                is_goal_state[next_state] = False
                
            visited_states[next_state] = new_cost

    return None, len(visited_states)



## Solve

In [58]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar,
    # "A*_optimal": solver_astar_optimal
}

In [59]:
def solve(map, method, heuristic = portal_deadlock_goals_heuristic, weight=1):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    if method == "A*":
        start_time = time.time()
        moves, numof_visited_states = SOLVERS[method](game_map, heuristic, weight)
        end_time = time.time()
        if weight == 1:
            print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
        else:
            print(f"{"weighted_" + method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    else:
        start_time = time.time()
        moves, numof_visited_states = SOLVERS[method](game_map)
        end_time = time.time()
        print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [60]:
def solve_all():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        for method in SOLVERS.keys():
            solve(map, method)
            

# Questions:

#### **1. What can be considered as the definition of a state in this problem? The most obvious definition for it is the warehouse map at each moment. What is the problem with that definition? How can it be improved?**


A state in this problem can be considered as the configuration that determines the game’s progress at any point. One definition would be the entire warehouse map at each moment, including all details. The problem with this is that fixed elements like goals, walls, and portals don’t change and would waste memory if stored for every state. Instead, the definition can be improved by focusing only on the variable elements: the player’s position and the boxes’ positions, which are the key factors that change and affect the solution, as implemented in the State class.  


#### **2. Based on the definition you have considered for the state, how would you define the actions?**

Given the state definition as the player’s position and the boxes’ positions, an action is defined as a move that changes the state by updating these positions. Specifically, an action is one of the four possible moves the player can make: moving up, down, left, or right, as defined in the Direction enum.
  
    
#### **3. Explain in detail how to model the problem, including the definition of actions, state, goal state, initial state, etc.**

To represent this game as a searchable problem (e.g., for AI algorithms like BFS, DFS, or A*), we need to define its key components: the state, actions, initial state, and goal state. Below, I’ll explain each in detail.

### State
A state describes the current configuration of the game at any point. It includes only the dynamic elements that change as the player moves:

- **Player’s Position**: Represented as a tuple (row, column) indicating where the player is on the grid.
- **Boxes’ Positions**: A tuple of (row, column) tuples, one for each box, tracking their locations.

Static elements like walls (impassable barriers), portals (teleportation points), and goal positions (where boxes need to end up) are not part of the state as explained previously.

### Actions
The player can take one of four actions at each step: move up, down, left, or right. However, the outcome of an action depends on the game’s rules:

- **Basic Movement**: If the adjacent cell in the chosen direction is empty or a portal, the player moves there.
    - If it’s a portal, the player teleports to the corresponding exit portal’s position.
- **Pushing a Box**: If the adjacent cell contains a box, the player can push it, but only if the cell beyond the box (in the same direction) is empty or a portal.
    - If the box is pushed into a portal, it teleports to the linked portal’s exit, and the player moves into the box’s original position.

### Initial State
The initial state is the starting point of the game, as provided by the game map. It specifies:

- The player’s starting position.
- The starting positions of all boxes.

### Goal State
The goal state is the desired end configuration:

- All boxes must be on their corresponding goal positions (specific (row, column) locations defined in the game map).

> ### Putting It Together
With these components, we can model the game as a state-space search problem:

1. Start at the initial state.
2. Use the actions to generate possible next states (e.g., moving the player, pushing boxes, handling portals).
3. Check each state against the goal state to see if the puzzle is solved.

Algorithms like BFS (breadth-first search), DFS (depth-first search), or A* (with a heuristic) can explore this space to find a sequence of moves from the initial state to the goal state.


#### **4. How can this problem be improved and the search space optimized? In search algorithms for this problem, should we expand all possible states that we can move to in the next step? What are the consequences of doing so? Explain what actions and considerations can be taken to improve this problem.**

### Optimizing the Search Space

The implementation enhances efficiency through several strategies:

- **Visited States Tracking**: A set is utilized to track states that have already been encountered, preventing redundant exploration and keeping the search space streamlined.
- **Early Goal Checking**: Each newly generated state is immediately evaluated to determine if it represents a goal. This avoids adding unnecessary states to the queue or stack.


Expanding every possible state is not practical for the following reasons:

- It increases memory consumption by including duplicate states.
- It delays the search process, especially in uninformed strategies like breadth-first search (BFS), where efficiency depends on minimizing unnecessary expansions.

Evaluating whether a state is a goal immediately after its generation accelerates the search due to these factors:

- It reduces memory usage by avoiding the storage of states that do not contribute to the solution.
- In methods such as depth-first search (DFS) or A*, this technique can yield faster results if a goal state is encountered early in the exploration.

These optimizations contribute to a search process that is both faster and more memory-efficient.


#### **5. Explain each of the implemented algorithms, highlighting the differences and advantages of each compared to the others. Mention which algorithms produce an optimal solution.**

- **BFS (Breadth-First Search)**: This algorithm explores all possible states level by level, starting from the initial state. It checks every move one step away, then two steps, and so on. Because of this, BFS always finds the shortest path to the goal, making it **optimal**. Its advantage is this guarantee of optimality, but it uses a lot of memory since it stores all states at each level.

- **DFS (Depth-First Search)**: DFS goes deep into one path until it hits a dead end, then backtracks to try another path. It uses less memory than BFS because it only tracks the current path. However, it’s **not optimal**. It might find a long path instead of the shortest one. Its advantage is simplicity and low memory use.

- **IDS (Iterative Deepening Search)**: IDS runs DFS multiple times, starting with a small depth limit (here 0) and increasing it until a solution is found. It combines DFS’s low memory use with BFS’s guarantee of the shortest path, making it **optimal**. Its advantage is balancing memory efficiency and optimality.

- **A\* Search**: Uses a heuristic to prioritize paths that seem closer to the goal. It’s faster than BFS or IDS in many cases and is **optimal** if the heuristic is admissible. Its advantage is efficiency guided by smart guesses but A bad heuristic can make A* inefficient.

**Differences and Advantages**:
- BFS is memory-heavy but optimal; DFS is memory-light but not optimal.
- IDS is a middle ground, optimal like BFS with less memory like DFS.
- A* is faster and optimal, beating others with a good heuristic but it stores all explored nodes, making it memory-intensive for large graphs.


> Problem with DFS and Why It’s Still Used:
  It can miss the shortest path and get stuck exploring deep, useless paths.
  But it uses little memory and can find a solution fast if the goal isn’t too deep, plus it’s easy to implement.


> How IDS Combines DFS and BFS
  IDS uses DFS’s deep exploration but repeats it with growing depth limits until the solution is found.
  It tackles BFS’s high memory use while keeping the shortest-path guarantee, blending DFS’s efficiency with BFS’s optimality.


#### **6 & 9. Implement the mentioned algorithms and justify your answers to question 5 with your implementations.**


In [61]:
for map in range(min(map_file_indices), max(map_file_indices) + 1):
    for method in SOLVERS.keys():
        if method == "BFS" and map == 9:
            continue
        if method == "DFS":
            if map in [6, 7 ,8, 9 ,10]:
                continue
        if method == "IDS":
            if map in [6, 7, 9, 10]:
                continue
        if method == "A*":
            if map == 9:
                solve(map, method, portal_deadlock_goals_heuristic, 2.5)
            else:
                solve(map, method, portal_deadlock_goals_heuristic, 1)
                solve(map, method, portal_deadlock_goals_heuristic, 2.5)
        else:
            solve(map, method)

BFS took 0.0 seconds on map 1 and visited 43 states.
7 moves were used: UDDULRR
DFS took 0.0 seconds on map 1 and visited 13 states.
7 moves were used: RLLRDUU
IDS took 0.0 seconds on map 1 and visited 158 states.
7 moves were used: RLLRDUU
A* took 0.0 seconds on map 1 and visited 22 states.
7 moves were used: DULRUDR
weighted_A* took 0.0 seconds on map 1 and visited 14 states.
7 moves were used: RLLRDUU
BFS took 0.0 seconds on map 2 and visited 20 states.
6 moves were used: LUDDUL
DFS took 0.0 seconds on map 2 and visited 9 states.
6 moves were used: LLRDUU
IDS took 0.0 seconds on map 2 and visited 62 states.
6 moves were used: LLRDUU
A* took 0.0 seconds on map 2 and visited 14 states.
6 moves were used: LUDDUL
weighted_A* took 0.0 seconds on map 2 and visited 10 states.
6 moves were used: LLRDUU
BFS took 0.0 seconds on map 3 and visited 121 states.
13 moves were used: ULDDUUUURDDDD
DFS took 0.0 seconds on map 3 and visited 19 states.
13 moves were used: ULDDUUUURDDDD
IDS took 0.01 se

### **Performance and Justification of Algorithms**

1. **BFS (Breadth-First Search)**  
   - **Finds optimal solutions** since it explores states level by level.  
   - **High memory usage**, as it stores all visited states at each level.  
   - **Slow for large maps** (e.g., Map 7 took 14.82s and visited 644,753 states).  

2. **DFS (Depth-First Search)**  
   - **Not optimal**, as it explores deep paths before checking shorter alternatives.  
   - **Low memory usage**, but sometimes finds unnecessarily long solutions.  
   - **Fast but inconsistent**, depending on the structure of the search space.  

3. **IDS (Iterative Deepening Search)**  
   - **Optimal like BFS** but **more memory-efficient**.  
   - **Slow due to repeated DFS calls** (e.g., 1.46s for Map 5 vs. 0.1s for BFS).  
   - Works well for small maps but inefficient for larger ones.  

4. **A\* Search**  
   - **Optimal if the heuristic is admissible**.  
   - **Faster than BFS** due to heuristic guidance (e.g., Map 7 took 0.3s vs. BFS's 14.82s).  
   - **Memory-intensive** but effective for practical use.  

5. **Weighted A\***  
   - **Not always optimal**, as it sacrifices correctness for speed.  
   - **Significantly faster than A\*** (e.g., Map 10 took 0.06s vs. 0.94s for A\*).  
   - **Useful for large-scale problems** where an approximate solution is acceptable.  

### **Conclusion**  
- **BFS and IDS guarantee optimality** but are impractical for large maps due to high memory usage.  
- **DFS is memory-efficient but unreliable for optimal solutions**.  
- **A\* is the best balance of efficiency and optimality**.  
- **Weighted A\* is the fastest** but does not always find the shortest solution.  

For real-world applications, **A\*** is often the best choice due to its speed and optimality. 

#### **7. Explain the heuristics introduced in the informed search section and analyze them in terms of admissibility and consistency.** 

In the informed search section, we introduced two heuristic functions for guiding the A* algorithm: **Manhattan Heuristic** and **Portal Deadlock Goals Heuristic**. Below, we analyze their admissibility and consistency.  

##### **1. Manhattan Heuristic**  

The **Manhattan heuristic** calculates the sum of Manhattan distances from each box to its corresponding goal, plus the minimum Manhattan distance from the player to any box.  

- **Not admissible**: This heuristic overestimates the actual cost when portals are present. Since portals can significantly reduce the path length, the Manhattan distance (which ignores them) may assign a cost higher than the optimal path cost.  
- **Not necessarily consistent**: If a state transition occurs via a portal, the estimated cost difference between states may not always match the true step cost, violating consistency.  

##### **2. Portal Deadlock Goals Heuristic**  

This heuristic improves upon the Manhattan heuristic by incorporating **portals, walls, and goal positions**. It calculates the shortest portal-adjusted distance from each box to its goal and considers player movement. Additionally, it penalizes boxes near walls (to prevent deadlocks) and rewards boxes already placed on goals.  

- **Admissible**: This heuristic never overestimates the true cost because it considers all possible paths, including those using portals, ensuring the estimated cost does not exceed the optimal solution cost.  
- **Likely consistent**: Since each state transition cost is properly reflected in the heuristic, and heuristic values decrease as boxes approach goals, it tends to satisfy consistency.   




#### **8. Run the algorithm using all the heuristics you introduced and compare their results with each other:**

In [62]:
for map in range(min(map_file_indices), max(map_file_indices) + 1):
    if map == 9:
        solve(map, "A*", portal_deadlock_goals_heuristic, 2.5)
    else:
        print("manhattan_heuristic:")
        solve(map, "A*", manhattan_heuristic, 1) 
        print("portal_deadlock_goals_heuristic:")
        solve(map, "A*", portal_deadlock_goals_heuristic, 1)


manhattan_heuristic:
A* took 0.0 seconds on map 1 and visited 43 states.
7 moves were used: DURLUDL
portal_deadlock_goals_heuristic:
A* took 0.0 seconds on map 1 and visited 22 states.
7 moves were used: DULRUDR
manhattan_heuristic:
A* took 0.0 seconds on map 2 and visited 20 states.
6 moves were used: LDUUDL
portal_deadlock_goals_heuristic:
A* took 0.0 seconds on map 2 and visited 14 states.
6 moves were used: LUDDUL
manhattan_heuristic:
A* took 0.0 seconds on map 3 and visited 93 states.
14 moves were used: ULUURDDDDUULDD
portal_deadlock_goals_heuristic:
A* took 0.0 seconds on map 3 and visited 58 states.
14 moves were used: ULUURDDDDUULDD
manhattan_heuristic:
A* took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
portal_deadlock_goals_heuristic:
A* took 0.0 seconds on map 4 and visited 2 states.
No Solution Found!
manhattan_heuristic:
A* took 0.03 seconds on map 5 and visited 1015 states.
15 moves were used: LULDLRDRDLLULUU
portal_deadlock_goals_heuristic:
A* took 0.0